# SF-DAT-21 | Unit Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in Unit Projects 1 and 2.

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pylab as pl
from sklearn import linear_model, cross_validation
%matplotlib inline

In [27]:
df_raw = pd.read_csv("../../dataset/admissions.csv")
df = df_raw.dropna()
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### Question 1. Let's create a frequency table of our variables.

In [88]:
# frequency table for prestige and whether or not someone was admitted
admitted_prestige = pd.crosstab(index=df["admit"], 
                            columns=df["prestige"])

admitted_prestige.columns = ["prestige1","prestige2","prestige3","prestige4"]
admitted_prestige.index= ["not_admit","admit"]

admitted_prestige

,prestige1,prestige2,prestige3,prestige4
not_admit,28,95,93,55
admit,33,53,28,12


## Part 2. Return of dummy variables

#### Question 2.1. Create class or dummy variables for prestige.

In [86]:
prestige_df = pd.get_dummies(df.prestige, prefix = 'prestige')

In [30]:
prestige_df.columns

Index([u'prestige_1.0', u'prestige_2.0', u'prestige_3.0', u'prestige_4.0'], dtype='object')

In [85]:
prestige_df.columns = ['prestige_1', 'prestige_2', 'prestige_3', 'prestige_4']

#### Question 2.2. When modeling our class variables, how many do we need?

Answer: We need just three classes, the fourth class will be the reference group.

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [32]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(prestige_df.ix[:, 'prestige_1':])
print handCalc.head()

   admit    gre   gpa  prestige_1  prestige_2  prestige_3  prestige_4
0      0  380.0  3.61         0.0         0.0         1.0         0.0
1      1  660.0  3.67         0.0         0.0         1.0         0.0
2      1  800.0  4.00         1.0         0.0         0.0         0.0
3      1  640.0  3.19         0.0         0.0         0.0         1.0
4      0  520.0  2.93         0.0         0.0         0.0         1.0


In [33]:
# crosstab prestige 1 admission
# frequency table cutting prestige and whether or not someone was admitted
admitted_prestige1 = pd.crosstab(index=handCalc["admit"], 
                            columns=handCalc["prestige_1"])
admitted_prestige1


prestige_1,0.0,1.0
admit,,
0,243,28
1,93,33


#### Question 3.1. Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college.

In [60]:
odds1 = 33.0/28.0
odds1

1.1785714285714286

#### Question 3.2. Now calculate the odds of admission if you did not attend a #1 ranked college.

In [61]:
odds2 = 93.0/243.0
odds2

0.38271604938271603

#### Question 3.3. Calculate the odds ratio.

In [62]:
ratio1 = odds1/odds2
ratio1

3.079493087557604

#### Question 3.4. Write this finding in a sentenance:

Answer: The odds of being admitted to graduate school are three times higher if you attended rank 1 university

#### Question 3.5. Print the cross tab for prestige_4.

In [34]:
admitted_prestige4 = pd.crosstab(index=handCalc["admit"], 
                            columns=handCalc["prestige_4"])
admitted_prestige4

prestige_4,0.0,1.0
admit,,
0,216,55
1,114,12


#### Question 3.6. Calculate the OR.

In [91]:
odds2_1 = 12.0/55.0
print odds2_1
odds2_2 = 114.0/216.0
print odds2_2
ratio2 = odds2_1/odds2_2
print ratio2

0.218181818182
0.527777777778
0.413397129187


#### Question 3.7. Write this finding in a sentence.

Answer: The odds of being admitted to graduate school are 60% lower if you attended rank 4 university

## Part 4. Analysis

In [35]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(prestige_df.ix[:, 'prestige_2':])
print data.head()

   admit    gre   gpa  prestige_2  prestige_3  prestige_4
0      0  380.0  3.61         0.0         1.0         0.0
1      1  660.0  3.67         0.0         1.0         0.0
2      1  800.0  4.00         0.0         0.0         0.0
3      1  640.0  3.19         0.0         0.0         1.0
4      0  520.0  2.93         0.0         0.0         1.0


We're going to add a constant term for our Logistic Regression.  The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [63]:
# manually add the intercept
data['intercept'] = 1.0
data.head()

,admit,gre,gpa,prestige_2,prestige_3,prestige_4,intercept
0,0,380.0,3.61,0.0,1.0,0.0,1.0
1,1,660.0,3.67,0.0,1.0,0.0,1.0
2,1,800.0,4.00,0.0,0.0,0.0,1.0
3,1,640.0,3.19,0.0,0.0,1.0,1.0
4,0,520.0,2.93,0.0,0.0,1.0,1.0


#### Question 4.1. Set the covariates to a variable called train_cols.

In [37]:
data.columns

Index([u'admit', u'gre', u'gpa', u'prestige_2', u'prestige_3', u'prestige_4',
       u'intercept'],
      dtype='object')

In [45]:
names_X = ['gre', 'gpa', 'prestige_2', 'prestige_3', 'prestige_4', 'intercept']

def X_y(df):
    X = df[ names_X ]
    y = df.admit
    return X, y

train_cols, train_y = X_y(data)

#### Question 4.2. Fit the model.

In [76]:
model = sm.Logit(train_y, train_cols)
result = model.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### Question 4.3. Print the summary results.

In [77]:
print result.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Thu, 09 Jun 2016   Pseudo R-squ.:                 0.08166
Time:                        00:56:56   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
gre            0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa            0.7793      0.333      2.344      0.019         0.128     1.431
prestige_2    -0.6801      0.317     -2.146      0.0

#### Question 4.4. Calculate the odds ratios of the coeffincients and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params

        conf.columns = ['2.5%', '97.5%', 'OR']

In [80]:
np.exp(result.params) - 1

gre           0.002221
gpa           1.180027
prestige_2   -0.493452
prestige_3   -0.737808
prestige_4   -0.788475
intercept    -0.979284
dtype: float64

In [84]:
conf = result.conf_int()
conf['OR'] = result.params
conf.columns = ['2.5%', '97.5%', 'OR']
print np.exp(conf)

                2.5%     97.5%        OR
gre         1.000074  1.004372  1.002221
gpa         1.136120  4.183113  2.180027
prestige_2  0.272168  0.942767  0.506548
prestige_3  0.133377  0.515419  0.262192
prestige_4  0.093329  0.479411  0.211525
intercept   0.002207  0.194440  0.020716


#### Question 4.5. Interpret the OR of Prestige_2.

Answer: Graduation from the university with prestige level 2 decreases the odds of graduate school admission by 49.3%

#### Question 4.6. Interpret the OR of GPA.

Answer: A one unit increase in GPA increases the odds of graduate school admission by 118%

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values.  This will allow us to see how the predicted probability of admission increases/decreases across different variables.  First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa".  This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

### I didn't get this part - sorry..

In [73]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [74]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)

print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])

gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)

print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])

# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### Question 5.1. Recreate the dummy variables.

In [ ]:
# recreate the dummy variables

# keep only what we need for making predictions


#### Question 5.2. Make predictions on the enumerated dataset.

#### Question 5.3. Interpret findings for the last 4 observations.

Answer:

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.